In [7]:
import requests
from config import DATA_DOWNLOAD_URL
import pandas as pd
import io
import gzip
import numpy as np
from datetime import datetime

In [8]:
def get_market_year(row): #make a market year row in datas
    year = row['DATE'].year
    month = row['DATE'].month
    if month >= 9:
        market_year = f"{year}/{year + 1}"
    else:
        market_year = f"{year - 1}/{year}"
    return market_year

In [3]:
df = pd.DataFrame(columns=["DATE", "DEP", "RR", "TN", "TX", "TM"])#, "UM"])

In [ ]:
for i in range(95):
    print(i)
    url = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_previous-1950-2023_RR-T-Vent.csv.gz"
    response = requests.get(url)

    compressed_data = io.BytesIO(response.content)
    decompressed_data = gzip.GzipFile(fileobj=compressed_data)

    tmp = pd.read_csv(decompressed_data, sep=";")
    tmp["DATE"] = pd.to_datetime(tmp["AAAAMMJJ"], format="%Y%m%d") #string to date
    tmp['DEP'] = i+1
    tmp = tmp[["DATE", "DEP", "RR", "TN", "TX", "TM"]] #keep only, in order : Date, departement code, précipitations, temp min, temp max and moy temp
    df = pd.concat([df, tmp])
df.to_csv(f"{DATA_DOWNLOAD_URL}/raw_rr_tn_tx_tm-1950-2023.csv")

In [ ]:
#data for current market year
for i in range(95):
    print(i)
    url = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_latest-2024-2025_RR-T-Vent.csv.gz"
    response = requests.get(url)

    compressed_data = io.BytesIO(response.content)
    decompressed_data = gzip.GzipFile(fileobj=compressed_data)

    tmp = pd.read_csv(decompressed_data, sep=";")
    tmp["DATE"] = pd.to_datetime(tmp["AAAAMMJJ"], format="%Y%m%d") #string to date
    tmp['DEP'] = i+1
    tmp = tmp[["DATE", "DEP", "RR", "TN", "TX", "TM"]] #keep only, in order : Date, departement code, précipitations, temp min, temp max and moy temp
    df = pd.concat([df, tmp])
df.to_csv(f"{DATA_DOWNLOAD_URL}/raw_rr_tn_tx_tm-current.csv")

Make a mean file for x last year and fo each month, each departement

In [ ]:
df = pd.read_csv(f"{DATA_DOWNLOAD_URL}/raw_rr_tn_tx_tm-1950-2023.csv")
df["DATE"] = pd.to_datetime(df["DATE"])
df = df[df["DATE"] >= "2019-09-01"] #year to start, i will take arbitrary last 5 market years
df = df.drop(columns=["Unnamed: 0"])
df["MY"] = df.apply(get_market_year, axis=1)
df["MONTH"] = df["DATE"].dt.month
df["YEAR-MONTH"] = df["DATE"].dt.to_period("M")

In [13]:
temp = df[["TN", "TX", "TM", "DEP", "MONTH"]].groupby(["DEP", "MONTH"]).mean().reset_index()
meanDailyPrecip = df[["RR", "DEP", "DATE"]].groupby(["DEP", "DATE"]).mean().reset_index()
meanDailyPrecip["YEAR-MONTH"] = pd.to_datetime(meanDailyPrecip["DATE"]).dt.to_period("M")
sumMonthlyPrecip = meanDailyPrecip[["DEP", "RR", "YEAR-MONTH"]].groupby(["DEP", "YEAR-MONTH"]).sum().reset_index()
sumMonthlyPrecip["MONTH"] = sumMonthlyPrecip["YEAR-MONTH"].dt.month
precip = sumMonthlyPrecip[["MONTH", "RR", "DEP"]].groupby(["DEP", "MONTH"]).mean().reset_index()

In [14]:
weather = pd.merge(temp, precip, on=["DEP", "MONTH"], how="inner")

In [7]:
weather.to_csv("../YieldModel/data/forecast/currentYear/historic_mean_weather.csv")

Same for the current year

In [12]:
df = pd.read_csv(f"{DATA_DOWNLOAD_URL}/raw_rr_tn_tx_tm-current.csv")
df["DATE"] = pd.to_datetime(df["DATE"])
df = df.drop(columns=["Unnamed: 0"])
df = df[df["DATE"] >= "2024-09-01"]
df["MY"] = df.apply(get_market_year, axis=1)
df["MONTH"] = df["DATE"].dt.month
df["YEAR-MONTH"] = df["DATE"].dt.to_period("M")

temp = df[["TN", "TX", "TM", "DEP", "MONTH"]].groupby(["DEP", "MONTH"]).mean().reset_index()
meanDailyPrecip = df[["RR", "DEP", "DATE"]].groupby(["DEP", "DATE"]).mean().reset_index()
meanDailyPrecip["YEAR-MONTH"] = pd.to_datetime(meanDailyPrecip["DATE"]).dt.to_period("M")
sumMonthlyPrecip = meanDailyPrecip[["DEP", "RR", "YEAR-MONTH"]].groupby(["DEP", "YEAR-MONTH"]).sum().reset_index()
sumMonthlyPrecip["MONTH"] = sumMonthlyPrecip["YEAR-MONTH"].dt.month
precip = sumMonthlyPrecip[["MONTH", "RR", "DEP"]].groupby(["DEP", "MONTH"]).mean().reset_index()

C:\Users\alexl\AppData\Local\Temp\ipykernel_2968\732414382.py:1: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{DATA_DOWNLOAD_URL}/raw_rr_tn_tx_tm-current.csv")


In [13]:
currentWeather = pd.merge(temp, precip, on=["DEP", "MONTH"], how="inner")

In [14]:
currentWeather.head(12)

,DEP,MONTH,TN,TX,TM,RR
0,1,1,-0.345996,7.201643,3.253289,164.560000
1,1,2,1.130804,9.039062,4.476190,43.906667
2,1,3,2.385484,13.458669,7.526667,58.786667
3,1,4,5.412083,18.343333,11.664444,116.833333
4,1,5,8.299683,20.197143,13.854286,89.850000
5,1,9,10.424792,20.138958,14.705111,175.386667
6,1,10,9.366935,18.344153,13.221382,171.293333
7,1,11,3.049893,11.362955,6.764531,62.060000
8,1,12,-1.229032,5.622151,1.805300,127.660000
9,2,1,0.781360,6.295226,3.237215,126.800000


In [15]:
currentWeather.to_csv("../YieldModel/data/forecast/currentYear/current_mean_weather.csv")